## CAVEclient Annotation Table tutorial

Uploading your annotations as an annotation table is a great way to be able to store neurons of interest in the FANC EM dataset. Tables allow you to track your neurons through changes in the segmentation (whether that be from proofreading or new versions), it also allows you to share your work at the benefit of the greater FANC community. This tutorial provides a few examples on how to create and upload annotations, how to add new annotations or edit existing annotations, and how to delete annotations.

In [1]:
#importing helpful packages
from caveclient import CAVEclient
import pandas as pd
import numpy as np 

If you have not registered an email with CAVEclient, please do this first. You can find an example for this here: https://github.com/htem/FANC_auto_recon/blob/main/example_notebooks/FANC_Connectomics_General_Intro.ipynb

If you are already registered, you are all set!

In [3]:
client = CAVEclient()
dataset = 'fanc_production_mar2021'
client = CAVEclient(dataset)

### Creating A Table

It is very important to provide a meaningful description for your table. This table will be visible for the FANC community, so the more information the better. At minimum, please include:
- what the data represents, 
- how the data was collected (if automated, which method was used), 
- when it was created or if it comes from a particular publication/submission. 
- who is managing the table and the home lab.

In [5]:
description = ''

The following parameters are required to create the table, you will only need to run create_table() once for each new table, so please me sure the information is correct.

Schema refers to the format of the table, different schemas require different information. Please refer to this link to see the available schemas and the minimal required information: https://global.daf-apis.com/schema/views/

In [ ]:
#schema to be used for the table 
schema = ''

table_name = '' #e.g. motor_neuron_table_v0 

#a list of the resolution that these annotations were done in. 
#for reference, FANC image resolution is [4.3,4.3,45]
voxel_resolution = []

In [6]:
client.annotation.create_table(table_name=table_name,
                               schema_name=schema,
                              voxel_resolution=voxel_resolution,
                              description = description)

### Uploading Annotations

Now that you've created a table, you will want to upload annotations. The format of the table will vary depending on your schema but the caveclient functions will be the same. 

Using the stage tool is very helpful in making sure that your data is in the proper format. You can find more documentation here: https://caveclient.readthedocs.io/en/latest/guide/annotation.html#staged-annotations

In [7]:
#here we are using the same parameters we used to create the table above
stage = client.annotation.stage_annotations(table_name=table_name,
                                    schema_name = schema,
                                    table_resolution=voxel_resolution)

Here is a sample dataframe for the 'bound_tag' schema. Please make sure to change this depending on your data and chosen table schema

In [9]:
sample_df = pd.DataFrame()

pt_positions = [[ 13329, 114635,   1861],
                [ 63394, 114393,   2304],
                [ 11729, 116735,   1478],
                [ 63084, 116459,   2863],
                [ 59659, 115855,   2065]]

tags = ['MN_A101_T1L', 'MN_A101_T1R', 'MN_A102_T1L', 'MN_A102_T1R', 'MN_A103_T1R']

sample_df['pt_position'] = pt_positions
sample_df['tag'] = tags

sample_df

,pt_position,tag
0,"[13329, 114635, 1861]",MN_A101_T1L
1,"[63394, 114393, 2304]",MN_A101_T1R
2,"[11729, 116735, 1478]",MN_A102_T1L
3,"[63084, 116459, 2863]",MN_A102_T1R
4,"[59659, 115855, 2065]",MN_A103_T1R


In [ ]:
#Once our data is in the right format, we can add the dataframe to our staging object
stage.add_dataframe(sample_df)

In [ ]:
#This allows you to list out the staged annotation before uploading to the client
stage.annotation_list

If everything looks good, upload the annotations using the line below. Note that if it is successful, you will see a list of numbers that correspond to the ids of these newly updated annotations.

In [ ]:
client.annotation.upload_staged_annotations(stage)

### Adding New Annotations

Adding new annotations to an existing table is the same as above, you just skip create_table() and call your table of interest when setting up your stage object. Note that these annotations will be added as new rows to your table. 

### Updating Existing Annotations

Updating existing annotation is very similar to uploading new annotations except for two things:
- set the update parameter as True
- add an id column to your dataframe that should match the annotation you plan to update

In [8]:
update_stage = client.annotation.stage_annotations(table_name, update=True)

In [10]:
#sample dataframe, let's say we're updating the tag for two neurons, with ID 0 and 2 
update_df = pd.DataFrame()

pt_positions = [[ 13329, 114635,   1861],
                [ 11729, 116735,   1478]]
tags = ['MN_A104_T1L', 'MN_A105_T1L']
ids = [0,2]

update_df['pt_position'] = pt_positions
update_df['tag'] = tags
update_df['id'] = ids

update_df

,pt_position,tag,id
0,"[13329, 114635, 1861]",MN_A104_T1L,0
1,"[11729, 116735, 1478]",MN_A105_T1L,2


In [ ]:
update_stage.add_dataframe(update_df)

In [ ]:
update_stage.annotation_list

#### Note about IDs. 
As before, once uploaded you will see a list of the updated IDs. Note that these new IDs will be different to the original IDs you just input. This is because every time there is a change to an annotation, it gets a new ID. That said, the caveclient keeps track of these changes so you can always go back and check the history of a given annotation.

In [11]:
#update your annotations
client.annotation.upload_staged_annotations(update_stage)

### Delete Annotations

If you want to remove individual annotations, you can do so by specifying the id you want removed. Note that annotations that are deleted are labelled as non-valid (so you will not see them in future queries) but they are not deleted from the table itself. Let's say we want to remove the annotation in the sample df above from ID 1, we would do the following:

In [17]:
#a list of the ids to be removed
to_remove = [1]
table_name = ''

In [ ]:
#if successful, the ids you deleted will be printed below
client.annotation.delete_annotation(table_name=table_name, annotation_ids = to_remove)